In [1]:
import pandas as pd
import numpy as np 

# Data

- England LADs to UK Regions: https://geoportal.statistics.gov.uk/documents/ons::local-authority-district-december-2020-to-lau1-to-itl3-to-itl2-to-itl1-january-2021-lookup-in-united-kingdom-v2-1/about

In [66]:
regions = pd.read_excel("data/raw/LAD_lookups.xlsx")

regions = regions[["LAU121CD", "ITL121CD", "ITL121NM"]]
regions = regions.rename(columns={"LAU121CD": "area_code", "ITL121CD": "itl_code", "ITL121NM": "itl_name"})
regions

,area_code,itl_code,itl_name
0,E06000001,TLC,North East (England)
1,E06000004,TLC,North East (England)
2,E06000002,TLC,North East (England)
3,E06000003,TLC,North East (England)
4,E06000005,TLC,North East (England)
...,...,...,...
383,N09000004,TLN,Northern Ireland
384,N09000001,TLN,Northern Ireland
385,N09000007,TLN,Northern Ireland
386,N09000008,TLN,Northern Ireland


# Using the ONS Subnational Dataset

In [68]:
dfs = pd.read_excel("data/raw/ons_subnational.xlsx", None, skiprows=0)
cleanded_dfs = []
for df in dfs.values():
    # itterate down the rows until a row with 'Area Code' in the first column is found
    # this is the start of the data
    for i in range(len(df)):
        if df.iloc[i,0] == 'Area Code':
            df.columns = df.iloc[i]
            df = df.iloc[i+1:]
            cleanded_dfs.append(df)
            break

df = pd.concat(cleanded_dfs)

id_vars = ['Area Code', 'Country', 'Nation', 'Region',
       'County or Unitary Authority', 'Local Authority District', 'Lower 95% Confidence Interval',
       'Upper 95% Confidence Interval', 'Notes', 'ITL Level 1',
       'ITL Level 2', 'ITL Level 3', 
       'Data accuracy']


df = df.melt(id_vars=id_vars, var_name='series', value_name='Value')
df = df.rename(columns={"Area Code":"area_code", "Value": "value"})


giving series codes

In [69]:
series = ['Gross Value Added (GVA) per hour worked (£)', 'Gross median weekly pay (£)', 'Combined Authority or City Region', 'Employment rate, aged 16 to 64 years (%)', 'Modelled unemployment rate, aged 16 years and over (%)', 'Gross disposable household income, per head (£)', 'Total value of UK exports (£ million)', 'Total FDI international investment position in the UK at end period (£ million)', 'Total FDI international investment position abroad at end period (£ million)', 'Total UK public-funded gross regional capital and non-capital expenditure on research and development (£ million)', 'Average travel time in minutes to reach nearest large employment centre (500 to 4999 jobs available), by public transport or walking (minutes)', 'Average travel time in minutes to reach nearest large employment centre (500 to 4999 jobs available), by car (minutes)', 'Average travel time in minutes to reach nearest large employment centre (500 to 4999 jobs available), by cycle (minutes)', 'Percentage of premises with gigabit-capable broadband (%)', 'Percentage of 4G coverage by at least one mobile network operator (%)', 'Percentage of pupils in state-funded schools meeting the expected standard in reading, writing and maths at the end of key stage 2 (%)', 'Observation Status', 'Percentage of young people achieving GCSEs (and equivalent qualifications) in English and maths by age 19 (%)', 'Percentage of schools rated good or outstanding by Ofsted (%)', 'Percentage of persistent absences (10% or more missed) for all pupils (%)', 'Percentage of persistent absences (10% or more missed) eligible for free school meals in the past 6 years (%)', 'Percentage of persistent absences (10% or more missed) for pupils who are looked after continuously for at least 12 months by local authorities (%)', "Percentage of 5-year olds at 'expected level' in communication and language early learning goals (%)", "Percentage of 5-year olds at 'expected level' in literacy early learning goals (%)", "Percentage of 5-year olds at 'expected level' in maths early learning goals (%)", 'Count of 19+ Further Education and Skills Learner Achievements (qualifications)', 'Apprenticeships started by adults aged 16+ based on home address (per 100,000 population)', 'Apprenticeships achieved by adults aged 16+ based on home address (per 100,000 population)', 'Proportion of the population aged 16-64 with NVQ3+ qualification (%)', '19+ further education and skills participation (per 100,000 population)', 'Welsh Health Board', 'Female Healthy Life Expectancy (years)', 'Male Healthy Life Expectancy (years)', 'Percentage of adults that currently smoke cigarettes (%)', 'Proportion of children living with obesity at reception age (%)', 'Proportion of children living with obesity at Year 6 age (%)', 'Proportion of adults living with obesity, aged 18 years and over (%)', 'Proportion of cancers diagnosed at stages 1 and 2 (%)', 'Age-standardised mortality rate for those aged under 75 (per 100,000 population)', 'Mean satisfaction with your life nowadays scored 0 (not at all) - 10 (completely)', 'Mean feeling things done in life are worthwhile scored 0 (not at all) - 10 (completely)', 'Mean happiness yesterday scored 0 (not at all) - 10 (completely)', 'Mean anxiety yesterday scored 0 (not at all) - 10 (completely)', 'Net additions per 1,000 stock', 'Police Force Area', 'Homicide Offences (per million population)', '% living in an area that has a devolution deal with a directly elected mayor '] # Freezing this list to make sure it won't change with updates to the data - Lazy   
series_code = pd.DataFrame({'series': series, 'series_code': range(1, len(series)+1)})
series_code.to_json("data/cleaned/series_codes.json", orient='records')

df = df.merge(series_code, on='series', how='left')


adding parent region codes

In [71]:
# adding England's
df = df.merge(regions,  on='area_code', how='left')
df

,area_code,Country,Nation,Region,County or Unitary Authority,Local Authority District,Lower 95% Confidence Interval,Upper 95% Confidence Interval,Notes,ITL Level 1,ITL Level 2,ITL Level 3,Data accuracy,series,value,series_code,itl_code,itl_name
0,E06000047,NaN,NaN,NaN,County Durham,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gross Value Added (GVA) per hour worked (£),30.64,1,TLC,North East (England)
1,E06000005,NaN,NaN,NaN,Darlington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gross Value Added (GVA) per hour worked (£),28.56,1,TLC,North East (England)
2,E06000001,NaN,NaN,NaN,Hartlepool,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gross Value Added (GVA) per hour worked (£),29.84,1,TLC,North East (England)
3,E06000002,NaN,NaN,NaN,Middlesbrough,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gross Value Added (GVA) per hour worked (£),29.5,1,TLC,North East (England)
4,E06000057,NaN,NaN,NaN,Northumberland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gross Value Added (GVA) per hour worked (£),30.43,1,TLC,North East (England)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598681,E12000005,NaN,NaN,West Midlands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,% living in an area that has a devolution deal...,48.975722,47,NaN,NaN
598682,E12000006,NaN,NaN,East of England,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,% living in an area that has a devolution deal...,14.126377,47,NaN,NaN
598683,E12000007,NaN,NaN,London,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,% living in an area that has a devolution deal...,100,47,NaN,NaN
598684,E12000008,NaN,NaN,South East,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,% living in an area that has a devolution deal...,0,47,NaN,NaN


## Making Series-UK data files

In [96]:
for Series in df.series.unique():
    Series_df = df[df.series == Series]
    Series_code = Series_df['series_code'].iloc[0]
    if str(series_code) == 'nan':
        continue

    # Make the UK-wide dataset
    UK_df = df.dropna(subset='ITL Level 1')
    UK_df = UK_df[["area_code", "ITL Level 1", "value"]].rename(columns={"Area Code": "area_code", "ITL Level 1": "name", "Value": "value"})
    UK_df = UK_df[UK_df.area_code.str.startswith("T")]
    #UK_df.to_json(f"data/cleaned/UK/series_{Series_code}.json", orient='records')


    # Make the regional datasets
    for itl_code in Series_df.itl_code.unique():
        itl_df = Series_df[Series_df.itl_code == itl_code]
        itl_df = itl_df.dropna(subset=["Local Authority District", "value"])
        if len(itl_df)==0:
            continue
        itl_df = itl_df[["area_code", "Local Authority District",  "value"]].rename(columns={"Local Authority District": "area_name", "value": "value"})
        # re-index to the area code
        itl_df = itl_df.set_index("area_code")
        itl_df = itl_df[~itl_df.index.duplicated(keep='first')]
        itl_df.to_json(f"data/cleaned/regional/series_{Series_code}_{itl_code}.json", orient='index')





In [93]:
df.head(5).to_json(orient=

'{"area_code":{"0":"E06000047","1":"E06000005","2":"E06000001","3":"E06000002","4":"E06000057"},"Country":{"0":null,"1":null,"2":null,"3":null,"4":null},"Nation":{"0":null,"1":null,"2":null,"3":null,"4":null},"Region":{"0":null,"1":null,"2":null,"3":null,"4":null},"County or Unitary Authority":{"0":"County Durham","1":"Darlington","2":"Hartlepool","3":"Middlesbrough","4":"Northumberland"},"Local Authority District":{"0":null,"1":null,"2":null,"3":null,"4":null},"Lower 95% Confidence Interval":{"0":null,"1":null,"2":null,"3":null,"4":null},"Upper 95% Confidence Interval":{"0":null,"1":null,"2":null,"3":null,"4":null},"Notes":{"0":null,"1":null,"2":null,"3":null,"4":null},"ITL Level 1":{"0":null,"1":null,"2":null,"3":null,"4":null},"ITL Level 2":{"0":null,"1":null,"2":null,"3":null,"4":null},"ITL Level 3":{"0":null,"1":null,"2":null,"3":null,"4":null},"Data accuracy":{"0":null,"1":null,"2":null,"3":null,"4":null},"series":{"0":"Gross Value Added (GVA) per hour worked (\\u00a3)","1":"Gros

In [90]:
itl_df

,area_code,Country,Nation,Region,County or Unitary Authority,Local Authority District,Lower 95% Confidence Interval,Upper 95% Confidence Interval,Notes,ITL Level 1,ITL Level 2,ITL Level 3,Data accuracy,series,value,series_code,itl_code,itl_name


In [80]:
Series_df[Series_df.area_name]

,area_code,Country,Nation,Region,County or Unitary Authority,Local Authority District,Lower 95% Confidence Interval,Upper 95% Confidence Interval,Notes,ITL Level 1,ITL Level 2,ITL Level 3,Data accuracy,series,value,series_code,itl_code,itl_name
585948,E06000047,NaN,NaN,NaN,County Durham,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,% living in an area that has a devolution deal...,NaN,47,TLC,North East (England)
585949,E06000005,NaN,NaN,NaN,Darlington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,% living in an area that has a devolution deal...,NaN,47,TLC,North East (England)
585950,E06000001,NaN,NaN,NaN,Hartlepool,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,% living in an area that has a devolution deal...,NaN,47,TLC,North East (England)
585951,E06000002,NaN,NaN,NaN,Middlesbrough,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,% living in an area that has a devolution deal...,NaN,47,TLC,North East (England)
585952,E06000057,NaN,NaN,NaN,Northumberland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,% living in an area that has a devolution deal...,NaN,47,TLC,North East (England)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598681,E12000005,NaN,NaN,West Midlands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,% living in an area that has a devolution deal...,48.975722,47,NaN,NaN
598682,E12000006,NaN,NaN,East of England,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,% living in an area that has a devolution deal...,14.126377,47,NaN,NaN
598683,E12000007,NaN,NaN,London,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,% living in an area that has a devolution deal...,100,47,NaN,NaN
598684,E12000008,NaN,NaN,South East,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,% living in an area that has a devolution deal...,0,47,NaN,NaN


In [78]:
itl_df

,name,value
586311,N09000001,NaN
586312,N09000002,NaN
586313,N09000003,NaN
586314,N09000004,NaN
586315,N09000005,NaN
...,...,...
598267,N09000007,NaN
598268,N09000008,NaN
598269,N09000009,NaN
598270,N09000010,NaN


In [36]:
UK_df.name.value_counts()

name
North East                  188
North West                  188
Yorkshire and The Humber    188
East Midlands               188
West Midlands               188
East of England             188
London                      188
South East (England)        188
South West (England)        188
Wales                       188
Scotland                    188
Northern Ireland            188
England                      47
Name: count, dtype: int64

In [19]:
df.Country.value_counts()

Country
United Kingdom       705
England and Wales    282
Great Britain        235
Name: count, dtype: int64

In [20]:
df.dropna(subset='ITL Level 1')['ITL Level 1'].value_counts()

ITL Level 1
North East                  188
North West                  188
Yorkshire and The Humber    188
East Midlands               188
West Midlands               188
East of England             188
London                      188
South East (England)        188
South West (England)        188
Wales                       188
Scotland                    188
Northern Ireland            188
UK not allocated             94
UK less Extra-Regio          47
England                      47
Name: count, dtype: int64